In [ ]:
%pip install ipynb python-dotenv transformers accelerate bitsandbytes kaggle kagglehub openpyxl transformers --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


: 

In [ ]:
from ipynb.fs.full.recommendations import *  # Roughly 90 seconds for 8GB VRAM GPU

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from ipynb.fs.full.emotion_classification_model import *

In [ ]:
# from ipynb.fs.full.completed_log_tracker_feature_extraction.ipynb import *  # Run this file in its original file to produce processed .csv file for Log Tracker

In [ ]:
# from ipynb.fs.full.completed_wellness_dataset_feature_extraction import * # Run this file in its original file to produce processed .csv file for wellness_dataset

In [ ]:
from Regressor_Predictive_Model import *

In [ ]:
import pandas as pd

# Load your dataset
daily_df = pd.read_csv("../data/processed/processed_log_tracker.csv")

# Columns that represent the tags
tag_columns = [
    'activity_level_grouping',
    'demographic_health_grouping',
    'sleep_health',
    'heart_health',
    'bp_health',
    'stress_profile_grouping',
    'social_wellness_grouping'
]

# Bundle the tags per row and removes any NaN/null values
daily_df['tags_per_row'] = daily_df[tag_columns].apply(
    lambda row: [tag for tag in row if pd.notna(tag)],
    axis=1
)

# Convert each list of tags into a comma-separated string
daily_df['tags_per_row_str'] = daily_df['tags_per_row'].apply(lambda tags: ', '.join(tags))

# View the result
print(daily_df[['tags_per_row_str']].head())

In [ ]:
from dotenv import load_dotenv
import os
import requests

# Load Environment Variables
load_dotenv()

# Hugging Face Token
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

#Weather API Token
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [ ]:
# Read the Initial Questions csv file into a DataFrame
initial_df = pd.read_csv('../data/raw/CS-GY-6923 Log Tracker  - Initial Questions.csv')

# Extract the last row of the DataFrame
last_updated_entry = initial_df.iloc[-1]

# De-structure relevant information from features into variables
gender = last_updated_entry['Gender']
age = last_updated_entry['Age']
height = last_updated_entry['Height']
activities = last_updated_entry['Activities for Mental Health (such as therapy sessions or meditation) [Comma separated]']
quality_social_interactions = last_updated_entry['Quality or frequency of social interactions']
location = last_updated_entry['Location']

# print(gender)
# print(age)
# print(height)
# print(activities)
# print(quality_social_interactions)
# print(location)

In [ ]:
# Weather Prompt
from datetime import datetime

def fetch_weather_data(city_name, api_key):
    """
    Fetch current weather description from WeatherAPI for a given city.
    """
    url = f'http://api.weatherapi.com/v1/current.json?key={api_key}&q={city_name}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if 'current' in data and 'condition' in data['current']:
            return data['current']['condition']['text']
        else:
            print("Warning: Weather data structure is incomplete.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None
    
def fetch_past_weather_data(city_name, date, api_key):
    """
    Fetch current weather description from WeatherAPI for a given city.
    """   

    raw_date = date  # MM/DD/YYYY format
    date_obj = datetime.strptime(raw_date, '%m/%d/%Y')
    formatted_date = date_obj.strftime('%Y-%m-%d')
    
    url = f'http://api.weatherapi.com/v1/history.json?key={api_key}&q={city_name}&dt={formatted_date}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        print(data)

        if 'forecast' in data and 'forecastday' in data['forecast'] and len(data['forecast']['forecastday']) > 0 and 'day' in data['forecast']['forecastday'][0] and 'condition' in data['forecast']['forecastday'][0]['day']:
            return data['forecast']['forecastday'][0]['day']['condition']['text']
        else:
            print("Warning: Weather data structure is incomplete.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None


def classify_weather_situation(weather_description):
    """
    Classify weather condition into one of: SUNNY, RAINY, CLOUDY, WINDY, or UNKNOWN.
    """
    description = weather_description.lower()

    if 'sunny' in description:
        return 'SUNNY'
    elif 'rain' in description or 'shower' in description:
        return 'RAINY'
    elif 'overcast' in description or 'cloud' in description:
        return 'CLOUDY'
    elif 'wind' in description:
        return 'WINDY'
    else:
        return 'UNKNOWN'

In [ ]:
# Preferences part of prompt
preferences = activities

In [ ]:
# Resulting Classification probability output from Regressor_Predictive_Model File
import torch
import time

daily_activity_recommendation_list = []

features_selected = ["duration_of_sleep_hours", "quality_of_sleep_110", "level_of_stress_scale_110", "level_of_workload_scale_110"]
regressor_probs_list = Invoke_Regressor(daily_df, features_selected)

for index, row in daily_df.iterrows():
    if index > 0:
        break
    notes = row['notes']  # Get the note for this row
    date = row['date'] # Get the date for this row
    mental_body_state_tags = row['tags_per_row_str']

    regressor_probs = torch.Tensor(regressor_probs_list[index])
    
    # Run the function on each note
    result = run_emotion_pipeline(notes, regressor_probs)

    # print("Emotion Classification Result:")
    # print(f"  {notes}")
    # print(f"  Superclass: {result['superclass']}")
    # print(f"  Sub-emotion: {result['sub_emotion']} ({result['confidence']:.3f})")
    # print("  Top 5 Emotions:")
    # for label, prob in result["top_5_emotions"]:
    #     print(f"    - {label}: {prob:.4f}")

    superclass_emotion = result['superclass']
    sub_emotion = result['sub_emotion']
    mood_string = f"{superclass_emotion} - {sub_emotion}"

    weather_description = fetch_past_weather_data(location, date, WEATHER_API_KEY) 
    # time.sleep(1)
    weather_situation = classify_weather_situation(weather_description)

    daily_recommended_activities= generate_activity_suggestions(mood_string, weather_situation, mental_body_state_tags, preferences)

    generated_prompt_parts = daily_recommended_activities.split("[/INST]")

    generated_prompt_question = generated_prompt_parts[0]
    generated_prompt_answer = generated_prompt_parts[1]
    
    daily_activity_recommendation_list.append(f"{generated_prompt_question.split(".")[0] + ". " + generated_prompt_question.split(".")[1] }: \n"  
                                              f"{generated_prompt_answer}")

In [ ]:
for daily_activity_recommendation in daily_activity_recommendation_list:
    print(daily_activity_recommendation)
    print()